### 1. Import Packages

In [1]:
# Librerie di base
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Librerie per la Data Imputation
from sklearn.impute import KNNImputer

# Librerie per il Machine Learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
import sklearn.metrics as metrics
from sklearn.model_selection import KFold, train_test_split, cross_val_score

# Librerie per la Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV

# Librerie per la Features Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.decomposition import PCA


print('All packages successfully loaded')

All packages successfully loaded


### 2. Load Data & Peak Sheet

In [2]:
df = pd.read_excel('../../../data/ST000369.xlsx')

# Rimuoviamo le colonne che non ci servono
df.drop(columns=["Idx", "SampleID", "Class"], inplace=True)

# Visualizziamo le prime 5 righe del dataset
df.head()

,SampleType,HealthState,SmokingStatus,Sex,M1,M2,M3,M4,M5,M6,...,M172,M173,M174,M175,M176,M177,M178,M179,M180,M181
0,Plasma,Adenocarcinoma,Former,F,194,168,77,105985,118,15489,...,195,1606,300,108,7203,43,59,1009,383,68
1,Plasma,Adenocarcinoma,Former,F,215,143,154,100462,133,13534,...,184,364,1364,160,11910,82,99,136,1021,165
2,Plasma,Adenocarcinoma,Current,F,104,67,45,75301,94,7390,...,189,157,884,73,6372,59,35,115,530,72
3,Plasma,Adenocarcinoma,Current,M,360,642,82,42097,84,50943,...,320,1621,461,104,14011,69,86,845,1309,127
4,Plasma,Adenocarcinoma,Current,M,96,137,95,112346,168,41987,...,84,769,266,158,18140,40,87,1213,1037,149


### 2.1 Data Cleaning

Per quanto riguarda il data cleaning del dataset, sono necessarie due tipologie di operazioni sui dati: 
1. Alcuni valori della feature "*HealthState*" sono scritti incorrettamente, perciò dobbiamo effettuarne la correzione.
2. Tre features binarie sono espresse attraverso stringhe e non valori binari (0 e 1), perciò dobbiamo fare la sostituzione.

In [3]:
# Effettuiamo la correzione dei valori errati
df["HealthState"] = df["HealthState"].str.replace('Adenocarcnoma', 'Adenocarcinoma')

# Eliminiamo le righe con valori nulli all'interno delle colonne "HealthState" e "Sex"
df = df.dropna(subset=["HealthState", "Sex"])

# Convertiamo i valori di natura categorica in valori numerici
df['Output'] = df['HealthState'].apply(lambda x: 1 if x in ['Adenosquamous', 'Adenocarcinoma'] else 0)
df['SmokingStatus'] = df['SmokingStatus'].apply(lambda x: 1 if x in ['Current'] else 0)
df['Sex'] = df['Sex'].apply(lambda x: 1 if x in ['F'] else 0)

# Visualizziamo le prime 5 righe del dataset
df.head()

,SampleType,HealthState,SmokingStatus,Sex,M1,M2,M3,M4,M5,M6,...,M173,M174,M175,M176,M177,M178,M179,M180,M181,Output
0,Plasma,Adenocarcinoma,0,1,194,168,77,105985,118,15489,...,1606,300,108,7203,43,59,1009,383,68,1
1,Plasma,Adenocarcinoma,0,1,215,143,154,100462,133,13534,...,364,1364,160,11910,82,99,136,1021,165,1
2,Plasma,Adenocarcinoma,1,1,104,67,45,75301,94,7390,...,157,884,73,6372,59,35,115,530,72,1
3,Plasma,Adenocarcinoma,1,0,360,642,82,42097,84,50943,...,1621,461,104,14011,69,86,845,1309,127,1
4,Plasma,Adenocarcinoma,1,0,96,137,95,112346,168,41987,...,769,266,158,18140,40,87,1213,1037,149,1


### 3. Extract X & Y

In [4]:
X = df.drop(columns=['SampleType', 'HealthState', 'Output'])
X_features_names = X.columns
y = df.Output

### 3.1 Data Imputation

In [5]:
# KNNImputer
imputer_knn = KNNImputer(n_neighbors=1)
imputer_knn.fit(X)
X[:] = imputer_knn.fit_transform(X)
X_knn = X.copy()

/var/folders/dv/cqkd8q211lb6w_8jd__s86d40000gn/T/ipykernel_10469/856958982.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X[:] = imputer_knn.fit_transform(X)


### 3.2 Train-test Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_knn, y, test_size=0.2, random_state=42)

### 4. Initial Model Build

In [8]:
# Definiamo il modello CNN
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compiliamo il modello
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Addestriamo il modello
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
5/5 [==============================] - 0s 25ms/step - loss: 8673.2305 - accuracy: 0.5462 - val_loss: 658.0284 - val_accuracy: 0.5152
Epoch 2/10
5/5 [==============================] - 0s 5ms/step - loss: 1071.5084 - accuracy: 0.4692 - val_loss: 1200.9955 - val_accuracy: 0.4848
Epoch 3/10
5/5 [==============================] - 0s 5ms/step - loss: 991.8897 - accuracy: 0.5000 - val_loss: 461.4939 - val_accuracy: 0.5758
Epoch 4/10
5/5 [==============================] - 0s 5ms/step - loss: 410.2192 - accuracy: 0.6077 - val_loss: 391.8039 - val_accuracy: 0.6364
Epoch 5/10
5/5 [==============================] - 0s 5ms/step - loss: 318.8388 - accuracy: 0.6385 - val_loss: 359.7742 - val_accuracy: 0.6364
Epoch 6/10
5/5 [==============================] - 0s 5ms/step - loss: 229.3968 - accuracy: 0.6769 - val_loss: 151.3397 - val_accuracy: 0.6970
Epoch 7/10
5/5 [==============================] - 0s 5ms/step - loss: 132.3798 - accuracy: 0.7000 - val_loss: 141.1867 - val_accuracy: 0.6667
Ep

### 5. Initial Model Evaluation


In [9]:
# Utilizziamo il metodo "predict" per ottenere le probabilità delle classi
y_pred_1 = model.predict(X_test)
# Applichiamo di una soglia di decisione (ad esempio, 0.5) per ottenere le classi previste
y_pred = (y_pred_1 > 0.5).astype(int)

# Calcoliamo le metriche del modello appena addestrato
accuracy = metrics.accuracy_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)
f1 = metrics.f1_score(y_test, y_pred)
conf_matrix = metrics.confusion_matrix(y_test, y_pred)
roc_auc = metrics.roc_auc_score(y_test, y_pred)

# Stampiamo le metriche
print(f'Accuratezza: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')
print(f'Matrice di Confusione:\n{conf_matrix}')
print(f'ROC AUC: {roc_auc}')

2/2 [==============================] - 0s 1ms/step
Accuratezza: 0.6666666666666666
Precision: 0.7142857142857143
Recall: 0.5882352941176471
F1-Score: 0.6451612903225806
Matrice di Confusione:
[[12  4]
 [ 7 10]]
ROC AUC: 0.6691176470588236
